In [1]:
import pandas as pd
import sqlite3
import requests

In [2]:
# Replace 'your_json_url' with the actual URL of your JSON data
json_url ='https://www.data.act.gov.au/resource/emq2-8bc4.json'
response = requests.get(json_url)
json_data = response.json()


In [3]:
# Assuming the JSON data is a list of dictionaries
df = pd.json_normalize(json_data)


In [4]:
df.head()

,crash_id,crash_date,crash_time,severity,crash_type,pedestrians,pedestrian_casualties,reported_location,location.latitude,location.longitude,location.human_address
0,1152106,2017-01-23T00:00:00.000,16:00,Injury,Struck pedestrian (On Road),1,1,RATCLIFFE CRESCENT (CHALLINOR -> CHALLINOR),-35.23191,149.054575,NaN
1,1162345,2017-05-23T00:00:00.000,18:45,Injury,Struck pedestrian (On Road),1,1,CAPTAIN COOK/FRANKLIN INTERSECTION,-35.320601,149.134962,NaN
2,1152674,2016-09-05T00:00:00.000,10:29,Injury,Struck pedestrian (Off Road),1,1,BORROWDALE STREET (NORFOLK -> BORROWDALE),-35.342443,149.12875,NaN
3,1154539,2017-03-03T00:00:00.000,16:23,Injury,Struck pedestrian (On Road),1,1,BARRY/KINGSLEY INTERSECTION,-35.275373,149.125049,NaN
4,1155513,2017-02-15T00:00:00.000,22:55,Property Damage Only,Struck pedestrian (On Road),1,0,HALLORAN CLOSE (BOWMAN -> HALLORAN),-35.24957,149.070075,NaN


In [5]:
df.shape


(563, 11)

In [6]:
conn = sqlite3.connect('crash_database.db')
df.to_sql('crash', conn, if_exists='replace', index=False)
conn.close()

In [7]:
# Connect to the SQLite database
conn = sqlite3.connect('crash_database.db')
cursor = conn.cursor()

# Create a new table 'crash1' with the specified column types, including 'crash_id'
cursor.execute("""
    CREATE TABLE IF NOT EXISTS crash1 (
        crash_id INTEGER PRIMARY KEY,
        crash_date DATE,
        crash_time TEXT,
        severity TEXT,
        crash_type TEXT,
        pedestrians INTEGER,
        pedestrian_casualties INTEGER,
        reported_location TEXT,
        latitude REAL,
        longitude REAL
    )
""")
conn.commit()

# Insert data from the existing 'crash' table into the 'crash1' table
cursor.execute("""
    INSERT INTO crash1 (
        crash_id,
        crash_date,
        crash_time,
        severity,
        crash_type,
        pedestrians,
        pedestrian_casualties,
        reported_location,
        latitude,
        longitude
    )
    SELECT
        CAST(crash_id AS INTEGER),  -- Convert crash_id to INTEGER 
        crash_date,
        crash_time,
        severity,
        crash_type,
        pedestrians,
        pedestrian_casualties,
        reported_location,
        CAST([location.latitude] AS REAL) AS latitude,
        CAST([location.longitude] AS REAL) AS longitude
    FROM crash
""")
conn.commit()

# Close the database connection
conn.close()


In [12]:
# Connect to your SQLite database
conn = sqlite3.connect('crash_database.db')


In [13]:
# Execute SQL to update the crash_date column
update_sql = """
UPDATE crash1
SET crash_date = SUBSTR(crash_date, 1, 10)
"""
conn.execute(update_sql)

# Commit the changes
conn.commit()

# Close the database connection
conn.close()

In [14]:
# Connect to your SQLite database
conn = sqlite3.connect('crash_database.db')

# Retrieve a few sample crash_date values
query = "SELECT DISTINCT crash_date FROM crash1 LIMIT 5"
sample_dates = conn.execute(query).fetchall()

# Display the sample dates
for date in sample_dates:
    print(date[0])

# Close the database connection
conn.close()

2012-01-28
2012-02-14
2012-02-10
2012-02-11
2012-02-18
